In [1]:
from google.colab import drive
drive.mount('/root/gdrive')

Drive already mounted at /root/gdrive; to attempt to forcibly remount, call drive.mount("/root/gdrive", force_remount=True).


In [2]:
%cd ~/gdrive/MyDrive/'_ECE 570'/Project/ops/voxel-tf-ops/
%ls

/root/gdrive/MyDrive/_ECE 570/Project/ops/voxel-tf-ops
avg_vox/          cuda_includes/  MANIFEST.in     tf/
BUILD             gpu/            README.md       third_party/
build_pip_pkg.sh  LICENSE         setup.py        utils/
configure.sh      Makefile        terminal.ipynb  WORKSPACE


In [3]:
!mkdir -p /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include

In [3]:
!cp -r cuda_includes/*.h /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include
!ls /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include

cuda_fp16.h  cuda.h


In [ ]:
# TF 2.7 had 3 errors in tensor_types.h, downgraded to TF 2.6 and works!
!pip uninstall tensorflow
!pip install tensorflow==2.6.2

In [4]:
!python3 -c 'import tensorflow as tf; print(tf.__version__)' 

2.6.2


In [5]:
import tensorflow as tf
# from avg_vox.python.ops.avg_vox_ops import avg_voxelize_forward

tf.debugging.set_log_device_placement(True)
gpu_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


In [6]:
!make clean

rm -f avg_vox/python/ops/_avg_vox_ops.cu.o avg_vox/python/ops/_avg_vox_ops.so trilinear_devox/python/ops/_trilinear_devox_ops.cu.o trilinear_devox/python/ops/_trilinear_devox_ops.so 


In [7]:
from tensorflow.python.framework.errors_impl import NotFoundError

avg_vox_ops_so_path = './avg_vox/python/ops/_avg_vox_ops.so'
try:
  avg_vox_ops = tf.load_op_library(avg_vox_ops_so_path)
except NotFoundError as e:
  print('Successfully cleaned!')
else:
  print('Colab is still seeing old .so')

Successfully cleaned!


In [ ]:
!make vox_gpu_only

In [9]:
!make vox_op

nvcc -std=c++11 -c -o avg_vox/python/ops/_avg_vox_ops.cu.o avg_vox/cc/avg_vox_kernels.cu.cc  -I/usr/local/lib/python3.7/dist-packages/tensorflow/include -D_GLIBCXX_USE_CXX11_ABI=0 -DEIGEN_MAX_ALIGN_BYTES=64 -Iutils -D GOOGLE_CUDA=1 -x cu -Xcompiler -fPIC -DNDEBUG --expt-relaxed-constexpr
/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(337): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(347): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/absl/strings/string_view.h(529): warning: expression has no effect

/usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/platform/file_system.h(574): warning: overloaded virtual function "tensorflow::FileSystem::FilesExist" is only partially overridden in class "tensorflow::WrappedFileSystem"

/usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/platform/fi

In [6]:
!make vox_test

python avg_vox/python/ops/avg_vox_ops_test.py
2021-11-11 20:28:23.086741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 20:28:23.095186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 20:28:23.095786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] AvgVoxTest.test_avg_voxelize_forward
2021-11-11 20:28:23.099362: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU i

In [10]:
avg_vox_ops = tf.load_op_library(avg_vox_ops_so_path)
# avg_vox_ops = tf.load_op_library('/root/gdrive/My Drive/_ECE 570/Project/ops/voxel-tf-ops/avg_vox/python/ops/_avg_vox_ops.so')
avg_voxelize_forward = avg_vox_ops.avg_vox_forward

In [144]:
# [B, C, N] = [2, 5, 4]
features = tf.constant([
  [[1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]],
  [[1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]],
], dtype=tf.float32)
# [B, 3, N] = [2, 3, 4]
coords = tf.constant([
  [[1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]],
  [[1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]],
], dtype=tf.int32)
resolution = tf.constant(4)

# for i in range(1,5):
  # out, ind, cnt = avg_voxelize_forward(features, coords, tf.constant(i))
out, ind, cnt = avg_voxelize_forward(features, coords, resolution)

# print(f"Output with voxel resolution = {i}")
print("ind\n---")
print(ind)
print()

print("cnt\n---")
print(cnt)
print()

print("out\n---")
print(out)
print("\n")


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AvgVoxForward in device /job:localhost/replica:0/task:0/device:GPU:0
ind
---
tf.Tensor(
[[21 42 63 84]
 [21 42 63 84]], shape=(2, 4), dtype=int32)

cnt
---
tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]], shape=(2, 64), dtype=int32)

out
---
tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0